In [1]:
from data_load import load_games, load_query_games
from utils import get_message_with_example, get_response, get_message_with_example_and_function_defn, get_fn_defn, GPT_model
import os

In [2]:
games = load_games()

In [3]:
import random
random.seed(40)

random_game_id = random.randint(0, len(games) - 1)
random_game = games[random_game_id]

other_games = games[:random_game_id] + games[random_game_id + 1:]

# choose 9 examples from other games
examples = random.sample(other_games, 9)

print(random_game)


Name: Epelle
Description: 3x3 intersecting lines with diagonals. Play occurs on the intersections of the lines. Players each have three pieces, initially placed on the sides closest to the player. Players move the pieces to an adjacent unoccupied intersection along the lines. The goal is to make three in a row which cannot be in the starting position.
Code: (define "NotAllPieceInHome" (not (all Sites (sites Occupied by:Mover) if:(is In (site) (sites Mover)))))

//------------------------------------------------------------------------------

(game "Epelle"
    (players 2) 
    (equipment { 
        ("AlquerqueBoard" 3)
        (piece "Marker" Each "StepToEmpty")
        (regions P1 (sites Bottom))
        (regions P2 (sites Top))
    }) 
    (rules 
        (start { 
            (place "Marker1" (sites P1)) 
            (place "Marker2" (sites P2)) 
        })
        (play (forEach Piece))
        
        (end 
            (if 
                (and 
                    "NotAllPieceIn

In [5]:
# predicted_code = get_response(get_message_with_example,random_game, examples)
predicted_code = get_fn_defn(random_game.code) + get_response(get_message_with_example_and_function_defn,random_game, examples)
actual_code = random_game.code

In [6]:

print("Predicted code:", predicted_code)
print("Actual code:", actual_code)

Predicted code: (define "NotAllPieceInHome" (not (all Sites (sites Occupied by:Mover) if:(is In (site) (sites Mover)))))

(game "Epelle"
    (players 2) 
    (equipment {
        ("AlquerqueBoard" 3 3)
        (piece "Marker" Each ("StepToEmpty" ~ (then ("If" "NotAllPieceInHome" ("ReplayIfLine3")))))
    })  
    (rules 
        (start {
            (place "Marker1" (sites Bottom))
            (place "Marker2" (sites Top))
        })
        (play (forEach Piece))
        (end ("Line3Win" Mover "NotAllPieceInHome"))
    )
)
Actual code: (define "NotAllPieceInHome" (not (all Sites (sites Occupied by:Mover) if:(is In (site) (sites Mover)))))

//------------------------------------------------------------------------------

(game "Epelle"
    (players 2) 
    (equipment { 
        ("AlquerqueBoard" 3)
        (piece "Marker" Each "StepToEmpty")
        (regions P1 (sites Bottom))
        (regions P2 (sites Top))
    }) 
    (rules 
        (start { 
            (place "Marker1" (sites P1)

In [12]:
# for each game, save the predicted code in predicted_codes directory
dir_path = f'{GPT_model}_predictions/descriptions'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
for i in range(len(games)):
    game = games[i]
    print(f"Processing game {game.tag}")
    other_games = games[:i] + games[i + 1:]
    examples = random.sample(other_games, 9)
    predicted_code = get_response(get_message_with_example, game, examples)
    save_path =  f'{dir_path}/{game.tag}.txt'
    # make new file with predicted code
    with open(save_path, 'w') as f:
        f.write(predicted_code)